<a href="https://colab.research.google.com/github/pachterlab/GRNP_2020/blob/master/notebooks/R_processing/ProcessR_Pooled.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Creates pooled data from several datasets to be used for prediction**

This notebook assembles CU histograms from multiple datasets for use in pooled prediction, a method in which lowly expressed genes can get a more stable CU histogram by "borrowing" information from similar datasets. We create two variants here - pooled data from full datasets and pooled data from downsampled dataset (downsampled 10 times). The reason for the latter is for use with down-sampling experiments, such as figure 4C. We generate pooled data from 6 PBMC 10X datasets. For the downsampled variant, we downsample each dataset 10 times and pool a total of 60 datasets, since so much information is lost in the downsampling.

Steps:
1. Clone the code repo and download data to process
2. Prepare the R environment
3. Process and save the data

The data used in this processing step is produced by the following notebooks:

Processing of FASTQ files with kallisto and bustools:

https://github.com/pachterlab/GRNP_2020/blob/master/notebooks/FASTQ_processing/ProcessEVALPBMC.ipynb
https://github.com/pachterlab/GRNP_2020/blob/master/notebooks/FASTQ_processing/ProcessPBMC_NG.ipynb
https://github.com/pachterlab/GRNP_2020/blob/master/notebooks/FASTQ_processing/ProcessPBMC_NG_2.ipynb
https://github.com/pachterlab/GRNP_2020/blob/master/notebooks/FASTQ_processing/ProcessPBMC_V2.ipynb
https://github.com/pachterlab/GRNP_2020/blob/master/notebooks/FASTQ_processing/ProcessPBMC_V3.ipynb
https://github.com/pachterlab/GRNP_2020/blob/master/notebooks/FASTQ_processing/ProcessPBMC_V3_2.ipynb
https://github.com/pachterlab/GRNP_2020/blob/master/notebooks/FASTQ_processing/ProcessPBMC_V3_3.ipynb

Preprocessing of BUG files:

https://github.com/pachterlab/GRNP_2020/blob/master/notebooks/R_processing/ProcessR_EVALPBMC.ipynb
https://github.com/pachterlab/GRNP_2020/blob/master/notebooks/R_processing/ProcessR_PBMC_NG.ipynb
https://github.com/pachterlab/GRNP_2020/blob/master/notebooks/R_processing/ProcessR_PBMC_NG_2.ipynb
https://github.com/pachterlab/GRNP_2020/blob/master/notebooks/R_processing/ProcessR_PBMC_V2.ipynb
https://github.com/pachterlab/GRNP_2020/blob/master/notebooks/R_processing/ProcessR_PBMC_V3.ipynb
https://github.com/pachterlab/GRNP_2020/blob/master/notebooks/R_processing/ProcessR_PBMC_V3_2.ipynb
https://github.com/pachterlab/GRNP_2020/blob/master/notebooks/R_processing/ProcessR_PBMC_V3_3.ipynb




**1. Clone the code repo and download data to process**

In [1]:
![ -d "GRNP_2020" ] && rm -r GRNP_2020

!git clone https://github.com/pachterlab/GRNP_2020.git


Cloning into 'GRNP_2020'...
remote: Enumerating objects: 274, done.
remote: Counting objects: 100% (274/274), done.
remote: Compressing objects: 100% (217/217), done.
remote: Total 1992 (delta 198), reused 83 (delta 57), pack-reused 1718
Receiving objects: 100% (1992/1992), 10.81 MiB | 18.44 MiB/s, done.
Resolving deltas: 100% (1383/1383), done.


In [2]:
#download BUG data from Zenodo
![ -d "figureData" ] && rm -r figureData
!mkdir figureData
!cd figureData && wget https://zenodo.org/record/4661263/files/PBMC_V3_3.zip?download=1 && unzip 'PBMC_V3_3.zip?download=1' && rm 'PBMC_V3_3.zip?download=1'

--2021-04-04 17:07:10--  https://zenodo.org/record/4661263/files/PBMC_V3_3.zip?download=1
Resolving zenodo.org (zenodo.org)... 137.138.76.77
Connecting to zenodo.org (zenodo.org)|137.138.76.77|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1819306744 (1.7G) [application/octet-stream]
Saving to: ‘PBMC_V3_3.zip?download=1’

PBMC_V3_3.zip?downl 100%[===================>]   1.69G  6.43MB/s    in 3m 13s  

2021-04-04 17:10:24 (9.01 MB/s) - ‘PBMC_V3_3.zip?download=1’ saved [1819306744/1819306744]

Archive:  PBMC_V3_3.zip?download=1
   creating: PBMC_V3_3/
  inflating: PBMC_V3_3/Bug_10.RData  
  inflating: PBMC_V3_3/Bug_100.RData  
  inflating: PBMC_V3_3/Bug_20.RData  
  inflating: PBMC_V3_3/Bug_40.RData  
  inflating: PBMC_V3_3/Bug_5.RData   
  inflating: PBMC_V3_3/Bug_60.RData  
  inflating: PBMC_V3_3/Bug_80.RData  
  inflating: PBMC_V3_3/ds_summary.txt  
  inflating: PBMC_V3_3/pooledHist.RData  
  inflating: PBMC_V3_3/pooledHistDS.RData  
  inflating: PBMC_V3_3/P

In [3]:
!cd figureData && wget https://zenodo.org/record/4661263/files/PBMC_V3_2.zip?download=1 && unzip 'PBMC_V3_2.zip?download=1' && rm 'PBMC_V3_2.zip?download=1'

--2021-04-04 17:10:43--  https://zenodo.org/record/4661263/files/PBMC_V3_2.zip?download=1
Resolving zenodo.org (zenodo.org)... 137.138.76.77
Connecting to zenodo.org (zenodo.org)|137.138.76.77|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1558568885 (1.5G) [application/octet-stream]
Saving to: ‘PBMC_V3_2.zip?download=1’

PBMC_V3_2.zip?downl 100%[===================>]   1.45G  11.7MB/s    in 2m 21s  

2021-04-04 17:13:07 (10.5 MB/s) - ‘PBMC_V3_2.zip?download=1’ saved [1558568885/1558568885]

Archive:  PBMC_V3_2.zip?download=1
   creating: PBMC_V3_2/
  inflating: PBMC_V3_2/Bug_10.RData  
  inflating: PBMC_V3_2/Bug_100.RData  
  inflating: PBMC_V3_2/Bug_20.RData  
  inflating: PBMC_V3_2/Bug_40.RData  
  inflating: PBMC_V3_2/Bug_5.RData   
  inflating: PBMC_V3_2/Bug_60.RData  
  inflating: PBMC_V3_2/Bug_80.RData  
  inflating: PBMC_V3_2/ds_summary.txt  
  inflating: PBMC_V3_2/pooledHist.RData  
  inflating: PBMC_V3_2/pooledHistDS.RData  
  inflating: PBMC_V3_2/P

In [4]:
!cd figureData && wget https://zenodo.org/record/4661263/files/PBMC_V3.zip?download=1 && unzip 'PBMC_V3.zip?download=1' && rm 'PBMC_V3.zip?download=1'

--2021-04-04 17:13:24--  https://zenodo.org/record/4661263/files/PBMC_V3.zip?download=1
Resolving zenodo.org (zenodo.org)... 137.138.76.77
Connecting to zenodo.org (zenodo.org)|137.138.76.77|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1127576758 (1.0G) [application/octet-stream]
Saving to: ‘PBMC_V3.zip?download=1’

PBMC_V3.zip?downloa 100%[===================>]   1.05G  12.5MB/s    in 95s     

2021-04-04 17:15:00 (11.3 MB/s) - ‘PBMC_V3.zip?download=1’ saved [1127576758/1127576758]

Archive:  PBMC_V3.zip?download=1
   creating: PBMC_V3/
  inflating: PBMC_V3/Bug_10.RData    
  inflating: PBMC_V3/Bug_100.RData   
  inflating: PBMC_V3/Bug_20.RData    
  inflating: PBMC_V3/Bug_40.RData    
  inflating: PBMC_V3/Bug_5.RData     
  inflating: PBMC_V3/Bug_60.RData    
  inflating: PBMC_V3/Bug_80.RData    
  inflating: PBMC_V3/ds_summary.txt  
  inflating: PBMC_V3/pooledHist.RData  
  inflating: PBMC_V3/pooledHistDS.RData  
  inflating: PBMC_V3/PredEvalData.RDS  
 

In [5]:
!cd figureData && wget https://zenodo.org/record/4661263/files/PBMC_NG.zip?download=1 && unzip 'PBMC_NG.zip?download=1' && rm 'PBMC_NG.zip?download=1'

--2021-04-04 17:15:11--  https://zenodo.org/record/4661263/files/PBMC_NG.zip?download=1
Resolving zenodo.org (zenodo.org)... 137.138.76.77
Connecting to zenodo.org (zenodo.org)|137.138.76.77|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1183970256 (1.1G) [application/octet-stream]
Saving to: ‘PBMC_NG.zip?download=1’

PBMC_NG.zip?downloa 100%[===================>]   1.10G  6.38MB/s    in 2m 10s  

2021-04-04 17:17:23 (8.68 MB/s) - ‘PBMC_NG.zip?download=1’ saved [1183970256/1183970256]

Archive:  PBMC_NG.zip?download=1
   creating: PBMC_NG/
  inflating: PBMC_NG/Bug_10.RData    
  inflating: PBMC_NG/Bug_100.RData   
  inflating: PBMC_NG/Bug_20.RData    
  inflating: PBMC_NG/Bug_40.RData    
  inflating: PBMC_NG/Bug_5.RData     
  inflating: PBMC_NG/Bug_60.RData    
  inflating: PBMC_NG/Bug_80.RData    
  inflating: PBMC_NG/ds_summary.txt  
  inflating: PBMC_NG/pooledHist.RData  
  inflating: PBMC_NG/pooledHistDS.RData  
  inflating: PBMC_NG/PredEvalData.RDS  
 

In [6]:
!cd figureData && wget https://zenodo.org/record/4661263/files/PBMC_NG_2.zip?download=1 && unzip 'PBMC_NG_2.zip?download=1' && rm 'PBMC_NG_2.zip?download=1'

--2021-04-04 17:17:35--  https://zenodo.org/record/4661263/files/PBMC_NG_2.zip?download=1
Resolving zenodo.org (zenodo.org)... 137.138.76.77
Connecting to zenodo.org (zenodo.org)|137.138.76.77|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1753743936 (1.6G) [application/octet-stream]
Saving to: ‘PBMC_NG_2.zip?download=1’

PBMC_NG_2.zip?downl 100%[===================>]   1.63G  9.40MB/s    in 3m 4s   

2021-04-04 17:20:41 (9.09 MB/s) - ‘PBMC_NG_2.zip?download=1’ saved [1753743936/1753743936]

Archive:  PBMC_NG_2.zip?download=1
   creating: PBMC_NG_2/
  inflating: PBMC_NG_2/Bug_10.RData  
  inflating: PBMC_NG_2/Bug_100.RData  
  inflating: PBMC_NG_2/Bug_20.RData  
  inflating: PBMC_NG_2/Bug_40.RData  
  inflating: PBMC_NG_2/Bug_5.RData   
  inflating: PBMC_NG_2/Bug_60.RData  
  inflating: PBMC_NG_2/Bug_80.RData  
  inflating: PBMC_NG_2/ds_summary.txt  
  inflating: PBMC_NG_2/pooledHist.RData  
  inflating: PBMC_NG_2/pooledHistDS.RData  
  inflating: PBMC_NG_2/P

In [7]:
!cd figureData && wget https://zenodo.org/record/4661263/files/PBMC_V2.zip?download=1 && unzip 'PBMC_V2.zip?download=1' && rm 'PBMC_V2.zip?download=1'

--2021-04-04 17:21:01--  https://zenodo.org/record/4661263/files/PBMC_V2.zip?download=1
Resolving zenodo.org (zenodo.org)... 137.138.76.77
Connecting to zenodo.org (zenodo.org)|137.138.76.77|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 816179326 (778M) [application/octet-stream]
Saving to: ‘PBMC_V2.zip?download=1’

PBMC_V2.zip?downloa 100%[===================>] 778.37M  17.0MB/s    in 83s     

2021-04-04 17:22:26 (9.35 MB/s) - ‘PBMC_V2.zip?download=1’ saved [816179326/816179326]

Archive:  PBMC_V2.zip?download=1
   creating: PBMC_V2/
  inflating: PBMC_V2/Bug_10.RData    
  inflating: PBMC_V2/Bug_100.RData   
  inflating: PBMC_V2/Bug_20.RData    
  inflating: PBMC_V2/Bug_40.RData    
  inflating: PBMC_V2/Bug_5.RData     
  inflating: PBMC_V2/Bug_60.RData    
  inflating: PBMC_V2/Bug_80.RData    
  inflating: PBMC_V2/ds_summary.txt  
  inflating: PBMC_V2/pooledHist.RData  
  inflating: PBMC_V2/pooledHistDS.RData  
  inflating: PBMC_V2/PredEvalData.RDS  
  in

In [8]:
!cd figureData && wget https://zenodo.org/record/4661263/files/EVALPBMC.zip?download=1 && unzip 'EVALPBMC.zip?download=1' && rm 'EVALPBMC.zip?download=1'


--2021-04-04 17:22:35--  https://zenodo.org/record/4661263/files/EVALPBMC.zip?download=1
Resolving zenodo.org (zenodo.org)... 137.138.76.77
Connecting to zenodo.org (zenodo.org)|137.138.76.77|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 467646609 (446M) [application/octet-stream]
Saving to: ‘EVALPBMC.zip?download=1’

EVALPBMC.zip?downlo 100%[===================>] 445.98M  12.8MB/s    in 39s     

2021-04-04 17:23:15 (11.3 MB/s) - ‘EVALPBMC.zip?download=1’ saved [467646609/467646609]

Archive:  EVALPBMC.zip?download=1
   creating: EVALPBMC/
  inflating: EVALPBMC/Bug_10.RData   
  inflating: EVALPBMC/Bug_100.RData  
  inflating: EVALPBMC/Bug_20.RData   
  inflating: EVALPBMC/Bug_40.RData   
  inflating: EVALPBMC/Bug_5.RData    
  inflating: EVALPBMC/Bug_60.RData   
  inflating: EVALPBMC/Bug_80.RData   
  inflating: EVALPBMC/ds_summary.txt  
  inflating: EVALPBMC/pooledHist.RData  
  inflating: EVALPBMC/pooledHistDS.RData  
  inflating: EVALPBMC/PredEvalData.R

In [9]:
#Check that download worked
!cd figureData && ls -l && cd PBMC_V3_2 && ls -l

total 28
drwxr-xr-x 2 root root 4096 Jul  1  2020 EVALPBMC
drwxr-xr-x 2 root root 4096 Jul  1  2020 PBMC_NG
drwxr-xr-x 2 root root 4096 Jul  1  2020 PBMC_NG_2
drwxr-xr-x 2 root root 4096 Jul  1  2020 PBMC_V2
drwxr-xr-x 2 root root 4096 Jul  1  2020 PBMC_V3
drwxr-xr-x 2 root root 4096 Jul  1  2020 PBMC_V3_2
drwxr-xr-x 2 root root 4096 Jul  1  2020 PBMC_V3_3
total 1628000
-rw-r--r-- 1 root root 352633388 Jun 30  2020 Bug_100.RData
-rw-r--r-- 1 root root 117036831 Jun 30  2020 Bug_10.RData
-rw-r--r-- 1 root root 187888424 Jun 30  2020 Bug_20.RData
-rw-r--r-- 1 root root 267712541 Jun 30  2020 Bug_40.RData
-rw-r--r-- 1 root root  65984760 Jun 30  2020 Bug_5.RData
-rw-r--r-- 1 root root 311852745 Jun 30  2020 Bug_60.RData
-rw-r--r-- 1 root root 335390002 Jun 30  2020 Bug_80.RData
-rw-r--r-- 1 root root      1065 Jul  1  2020 ds_summary.txt
-rw-r--r-- 1 root root    256079 Jul  1  2020 pooledHistDS.RData
-rw-r--r-- 1 root root    518870 Jul  1  2020 pooledHist.RData
-rw-r--r-- 1 root root  2

**2. Prepare the R environment**

In [10]:
#switch to R mode
%reload_ext rpy2.ipython


In [11]:
#install the R packages
%%R
install.packages("qdapTools")
install.packages("dplyr")
install.packages("stringdist")


R[write to console]: Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

R[write to console]: also installing the dependencies ‘bitops’, ‘chron’, ‘data.table’, ‘RCurl’, ‘XML’


R[write to console]: trying URL 'https://cran.rstudio.com/src/contrib/bitops_1.0-6.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 8734 bytes

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[writ

**3. Process and save the data**

Here we generate the pooled CU histograms, both from full datasets and downsampled ones. In addition, we calculate the average expression when downsampling 20 times, which will to a large extent remove the sampling noise. We then estimate the sampling noise as the difference between this mean and data that is downsampled only once.

In [12]:
#First set some path variables
%%R
source("GRNP_2020/RCode/pathsGoogleColab.R")


In [13]:
#Process and filter the BUG file
%%R
source(paste0(sourcePath, "ButterflyHelpers.R"))

#generate pooled histograms (for fig 3C)

loadBug("PBMC_V3")
generatePooledHistogramDS("PBMC_V3")
generatePooledHistogram("PBMC_V3") #these are not really used, but we generate them anyway, it is little extra effort
rmBug("PBMC_V3")

loadBug("PBMC_V3_2")
generatePooledHistogramDS("PBMC_V3_2")
generatePooledHistogram("PBMC_V3_2")
rmBug("PBMC_V3_2")

loadBug("PBMC_V3_3")
generatePooledHistogramDS("PBMC_V3_3")
generatePooledHistogram("PBMC_V3_3")
rmBug("PBMC_V3_3")

loadBug("PBMC_NG")
generatePooledHistogramDS("PBMC_NG")
generatePooledHistogram("PBMC_NG")
rmBug("PBMC_NG")

loadBug("PBMC_NG_2")
generatePooledHistogramDS("PBMC_NG_2")
generatePooledHistogram("PBMC_NG_2")
rmBug("PBMC_NG_2")

loadBug("PBMC_V2")
generatePooledHistogramDS("PBMC_V2")
generatePooledHistogram("PBMC_V2")
rmBug("PBMC_V2")

loadBug("EVALPBMC")
generatePooledHistogramDS("EVALPBMC")
generatePooledHistogram("EVALPBMC")
rmBug("EVALPBMC")



R[write to console]: 
Attaching package: ‘dplyr’


R[write to console]: The following objects are masked from ‘package:stats’:

    filter, lag


R[write to console]: The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




[1] "Preparing..."
[1] 1
[1] "1:1"
[1] "1:2"
[1] "1:3"
|===================================================== |100% ~0 s remaining     [1] "1:4"
[1] "1:5"
[1] 2
[1] "2:1"
[1] "2:2"
[1] "2:3"
|===================================================== |100% ~0 s remaining     [1] "2:4"
[1] "2:5"
[1] 3
[1] "3:1"
[1] "3:2"
[1] "3:3"
|===================================================== |100% ~0 s remaining     [1] "3:4"
[1] "3:5"
[1] 4
[1] "4:1"
[1] "4:2"
[1] "4:3"
|===================================================== |100% ~0 s remaining     [1] "4:4"
[1] "4:5"
[1] 5
[1] "5:1"
[1] "5:2"
[1] "5:3"
|===================================================== |100% ~0 s remaining     [1] "5:4"
[1] "5:5"
[1] 6
[1] "6:1"
[1] "6:2"
[1] "6:3"
|===================================================== |100% ~0 s remaining     [1] "6:4"
[1] "6:5"
[1] 7
[1] "7:1"
[1] "7:2"
[1] "7:3"
|===================================================== |100% ~0 s remaining     [1] "7:4"
[1] "7:5"
[1] 8
[1] "8:1"
[1] "8:2"
[1]

In [16]:
!cd figureData && ls -l && cd EVALPBMC && ls -l

total 28
drwxr-xr-x 2 root root 4096 Jul  1  2020 EVALPBMC
drwxr-xr-x 2 root root 4096 Jul  1  2020 PBMC_NG
drwxr-xr-x 2 root root 4096 Jul  1  2020 PBMC_NG_2
drwxr-xr-x 2 root root 4096 Jul  1  2020 PBMC_V2
drwxr-xr-x 2 root root 4096 Jul  1  2020 PBMC_V3
drwxr-xr-x 2 root root 4096 Jul  1  2020 PBMC_V3_2
drwxr-xr-x 2 root root 4096 Jul  1  2020 PBMC_V3_3
total 486724
-rw-r--r-- 1 root root 87322865 Jun 30  2020 Bug_100.RData
-rw-r--r-- 1 root root 53475778 Jun 30  2020 Bug_10.RData
-rw-r--r-- 1 root root 65711410 Jun 30  2020 Bug_20.RData
-rw-r--r-- 1 root root 75161084 Jun 30  2020 Bug_40.RData
-rw-r--r-- 1 root root 37818341 Jun 30  2020 Bug_5.RData
-rw-r--r-- 1 root root 80649419 Jun 30  2020 Bug_60.RData
-rw-r--r-- 1 root root 84316810 Jun 30  2020 Bug_80.RData
-rw-r--r-- 1 root root      992 Jul  1  2020 ds_summary.txt
-rw-r--r-- 1 root root   316261 Apr  4 19:09 pooledHistDS.RData
-rw-r--r-- 1 root root   720164 Apr  4 19:10 pooledHist.RData
-rw-r--r-- 1 root root 11259902 Jul 